This notebook shows how to set up a Python environment for the BART library on Google Colab.

In this project, BART is used to estimate the coil sensitivity maps via ESPIRiT calibration, in order to generate ground truth coil-combined MRI images from the given dataset of k-space volumes.

Running the following code before launching the train_test programs should be enough for things to work smoothly. If errors arise, please refer to the official BART documentation at:
- [BART Setup on Colab](https://github.com/mrirecon/bart-workshop/blob/master/ismrm2021/neural_networks/bart_neural_networks.ipynb)
- [BART Setup on local machine](https://mrirecon.github.io/bart/)

# Setup BART for Colab

In [ ]:
%%bash

# Use CUDA 10.1 when on Tesla K80

# Estimate GPU Type
GPU_NAME=$(nvidia-smi --query-gpu=gpu_name --format=csv,noheader)

echo "GPU Type:"
echo $GPU_NAME

if [ "Tesla K80" = "$GPU_NAME" ];
then
    echo "GPU type Tesla K80 does not support CUDA 11. Set CUDA to version 10.1."

    # Change default CUDA to version 10.1
    cd /usr/local
    rm cuda
    ln -s cuda-10.1 cuda
else
    echo "Current GPU supports default CUDA-11."
    echo "No further actions are necessary."
fi

echo "GPU Information:"
nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv
nvcc --version

GPU Type:
Tesla T4
Current GPU supports default CUDA-11.
No further actions are necessary.
GPU Information:
name, driver_version, memory.total [MiB]
Tesla T4, 460.32.03, 15109 MiB
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [ ]:
%%bash

# Install BARTs dependencies
apt-get install -y make gcc libfftw3-dev liblapacke-dev libpng-dev libopenblas-dev &> /dev/null

# Clone Bart
[ -d /content/bart ] && rm -r /content/bart
git clone https://github.com/mrirecon/bart/ bart &> /dev/null

In [ ]:
%%bash

cd bart

# For long term support, we checkout the following tag:
git checkout tags/ISMRM21_NN

# Define specifications 
COMPILE_SPECS=" PARALLEL=1
                CUDA=1
                CUDA_BASE=/usr/local/cuda
                CUDA_LIB=lib64
                OPENBLAS=1
                BLAS_THREADSAFE=1"

printf "%s\n" $COMPILE_SPECS > Makefiles/Makefile.local

make &> /dev/null

Note: checking out 'tags/ISMRM21_NN'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 5287d1c5 Add tool to transform onehotencoded data to integer encoded


In [ ]:
%env TOOLBOX_PATH=/content/bart

env: TOOLBOX_PATH=/content/bart


In [ ]:
import os
import sys

os.environ['PATH'] = os.environ['TOOLBOX_PATH'] + ":" + os.environ['PATH']
sys.path.append(os.environ['TOOLBOX_PATH'] + "/python/")